<a href="https://colab.research.google.com/github/allaalmouiz/deepLearning_stroke_prediction/blob/main/deepLearning_stroke_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stroke Risk Prediction Challange - Module 4
Submitted by: **`Alaa Almouiz F. Moh.`**

ID Number: **`S2026_176`**

Track: **Machine Learning**

For: **ZAKA ©**

## **1- Problem Statement (Objective)**

I’ve been asked to assist a public health organization in identifying individuals most at risk of having a stroke, using a dataset of patient information and health indicators.

So, I will build a Deep Learning **Binary Classification Model** to predicts whether a patient will experinece a stroke or not.


### **Dataset**
Stroke Risk Dataset (Downloaded  from Kaggle: [Data](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)), and been diwnloaded to [my repo](https://github.com/allaalmouiz/deepLearning_stroke_prediction/blob/739145a7bf7db1e6814682ae25a67beef25940fe/healthcare-dataset-stroke-data.csv).

The dataset variables include:
* `id`: Unique identifier for each patient.
* `gender`: Patient’s gender.
* `aga`: Age of the patient.
* `hypertension`:  if the patient has hypertension.
* `ever_married`: Marital status.
* `work_type`: Type of employment.
* `heart_disease`: if the patient has a history of heart disease.
* `Residence_type`: Patient’s area of residence.
* `avg_glucose_level`: Average blood glucose level.
* `bmi`: Body Mass Index.
* `smoking_status`: Patient’s smoking status.
* `stroke`Whether the patient experienced a stroke.
